# API를 이용해 ChatGPT API 활용하기

현재 Python이 실행 중인 가상환경과 일치하는지 확인하기 위해 실행하는 코드이다.   ㅌㅌㅌ
정상적이라면 /Path/가상환경명/bin/python과 같은 형태로 출력된다.

In [ ]:
!python --version

In [ ]:
import sys
sys.executable

## ChatGPT API 사용을 위해 미리 해야 할 일

<h3>1. ChatGPT API 키 발급받기</h3>
<h3>2. 가상환경 설치하기(anaconda)</h3>
Anaconda는 가상환경을 설치하고 지원하는 프로그램이며, 이를 설치하면 별도의 파이썬을 설치하지 않아도 된다.<br>
설치 링크 <a href="https://www.anaconda.com/download/">click</a>
<h3>3. 나만의 가상환경 만들기</h3>
명령 프롬프트나 터미널을 열어서 아래의 명령어를 입력한다.<br>
<b>conda create -n llm python=3.11.5</b><br>
<b>conda activate llm</b>
<b>pip install -r requirements.txt</b><br>
깃허브 레포지토리에서 다운로드 받으면 requirements.txt를 받을 수 있으며, 해당 폴더로 이동해 명령어를 실행해야 한다.<br>

In [ ]:
import os
import time
import re
import json
import openai
from openai._client import OpenAI #ChatGPT API 이용을 위한 라이브러리
from dotenv import load_dotenv #.env에서 파일을 읽어오기 위한 환경변수 처리 라이브러리

load_dotenv() # 환경 파일 불러오기

client = OpenAI(api_key = os.getenv('OPENAI_API_KEY'))

In [ ]:
TEMPERATURE = 0.0
MAX_TOKENS = 16000
MODEL = 'gpt-3.5-turbo-0125'
context = [] # 리스트

def check_tokens(items):
    cnt = 0

    if items is None:
        return cnt

    for item in items:
        cnt += len(item['content'])

    return cnt

## Chatbot

In [ ]:
def conversate():
    while(1):
        message = input('Chat:')
        message = message.strip()
    
        if message == '':
            print('Input your talk.')
            continue
        elif message == 'exit':
            break
    
        # Examine if the size of check is over the maximum tokens
        total_cnt = check_tokens(context) + len(message)

        if total_cnt >= MAX_TOKENS:
            context.clear()
            print('context cleared.')

        # Setup up for message to call ChatGPT
        if len(context) == 0:
            context.append({"role": "system", "content": "You are a helpful assistant."})
            context.append({"role": "user", "content": message})
        else:
            context.append({"role": "user", "content": message})

        response = client.chat.completions.create(model=MODEL, messages=context, temperature=TEMPERATURE)
        answer = response.choices[0].message.content
        print(f"AI: {answer}")
        #codes = markdown.markdown(answer, extensions=['fenced_code', 'codehilite'])
        context.append({'role': 'assistant', 'content': answer})

        if check_tokens(context) >= MAX_TOKENS:
            context.clear()

In [ ]:
message = '파이썬을 공부하려면 어떤 책을 읽으면 좋을까?'

messages = []
messages.append({"role": "system", "content": "너는 훌륭한 비서야."})
messages.append({"role": "user", "content": message})

response = client.chat.completions.create(model=MODEL, messages=messages, temperature=TEMPERATURE)
answer = response.choices[0].message.content
print(answer)

In [ ]:
messages

In [ ]:
print(answer)

In [ ]:
conversate()

In [ ]:
from IPython.display import display, Markdown
context = []

def single_ask(message):
    message = message.strip()

    if message == '':
        print('대화 내용을 입력하세요.')
    elif message == 'exit':
        return

    # 대화 맥락을 고려하여 전체 최대 토큰을 초과하는지 체크하도록 한다.
    total_cnt = check_tokens(context) + len(message)

    if total_cnt >= MAX_TOKENS:
        context.clear()
        print('context cleared.')

    # ChatGPT 대화를 위한 메시지 형태 설정하기
    if len(context) == 0:
        context.append({"role": "system", "content": "You are a helpful assistant."})
        context.append({"role": "user", "content": message})
    else:
        context.append({"role": "user", "content": message})

    response = client.chat.completions.create(model=MODEL, messages=context, temperature=TEMPERATURE)
    answer = response.choices[0].message.content
    display(Markdown(answer))  # 마크다운 형태로 해석하여 출력
    
    context.append({'role': 'assistant', 'content': answer})

    if check_tokens(context) >= MAX_TOKENS:
        context.clear()

In [ ]:
single_ask("1부터 100까지 소수를 찾는 파이썬 코드를 알려 줘.")

In [ ]:
single_ask("그래서 1부터 100까지 소수는 어떤 것들이 있어?")

In [ ]:
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

prime_numbers = [num for num in range(1, 101) if is_prime(num)]
print(prime_numbers)

In [ ]:
context = [] # 리스트
init_msg = '''
아래의 기사를 토대로 코로나19 백신에 대한 부작용에 대해 나와 토론하려고 해.
먼저 네가 "안녕. 너의 의견은 뭐니?"라고 시작해 줘.
서로 한 번씩 의견을 돌아가면서 제시하게 될 거야.
그리고 내가 자신의 의견을 말하면 너는 그것에 반대해서 계속해서 증거를 제시하면서 토론해.

기사: 코로나19 백신이 심장 관련 염증이나 뇌혈전 등의 부작용과 연관이 있다는 연구 결과가 나왔다. 
19일(이하 현지시간) 블룸버그에 따르면 글로벌 백신 데이터 네트워크(GVDN)가 코로나19 백신 접종과 관련해 소수가 신경계, 혈액, 심장 관련 부작용과 연관이 있다고 전했다. 
이 연구는 8개국 9900만명의 백신 접종자를 대상으로 진행됐다. 연구진은 접종자들에게서 코로나19 백신 주요 부작용으로 간주되는 13가지 질환을 집중 조사해 코로나 백신접종과 
부작용 간의 상관관계를 연구했다. 그 결과 심장 근육의 염증을 뜻하는 '심근염'은 mRNA(메신저 리보핵산) 백신을 1차~3차 접종한 환자들에게서 지속적으로 확인됐다. 
이 증상은 모더나 백신 주사를 두 번째 접종한 후 가장 많이 나타났다. '박제브리아'라 일컫는 아스트라제네카 백신은 길랑-바레 증후군과 연관이 있었다. 
이외 뇌혈전의 일종인 뇌정맥동 혈전증의 증가와도 연관이 있었다. 당초 연구진은 이 질환의 발병 건수를 66건으로 예상했지만 이를 상회하는 190건의 사례를 확인했다. 
블룸버그에 따르면 코로나19 백신은 지난 3년 동안 총 135억회 이상 투여됐다. 유럽에서만 100만명 이상의 고위험군 환자들이 코로나19 팬데믹 중에도 생명을 건질 수 있었다. 
그럼에도 백신 접종자 중 소수는 여전히 부작용을 호소하고 있는 만큼 코로나19 백신과 관련한 논쟁은 지속되고 있다.
'''
client = OpenAI(api_key = os.getenv('OPENAI_API_KEY'))

def debate():
    is_start = True
    
    while(1):
        if not is_start:
            message = input('Human: ')
            message = message.strip()
    
            if message == '':
                print('Input your text.')
                continue
            elif message == 'exit':
                break
    
            # 대화 맥락을 고려하여 전체 최대 토큰을 초과하는지 체크하도록 한다.
            total_cnt = check_tokens(context) + len(message)

            if total_cnt >= MAX_TOKENS:
                context.clear()
                print('context cleared.')
        else:
            is_start = False

        # ChatGPT 대화를 위한 메시지 형태 설정하기
        if len(context) == 0:
            context.append({"role": "system", "content": "너는 훌륭한 토론 선생님이야."})
            context.append({"role": "user", "content": init_msg})
        else:
            context.append({"role": "user", "content": message})

        response = client.chat.completions.create(model=MODEL, messages=context, temperature=TEMPERATURE)
        answer = response.choices[0].message.content
        print(f"AI: {answer}")
        
        context.append({'role': 'assistant', 'content': answer})
        
        if check_tokens(context) >= MAX_TOKENS:
            context.clear()

In [ ]:
debate()

## How to use ChatGPT for textual data analysis

In [ ]:
query = '''
보기는 학생이 작성한 글이야. 
툴민의 논증(Toulmin's Argumentation Pattern)에 따라 글에 포함되어 있는 주장(claim), 반박(rebuttal), 자료(data), 보장(warrant), 뒷받침(backing), 제한 조건(qualifer)이 명시적으로 포함되어 있는지 확인하고 추출해. 
만약 관련된 요소가 글 속에 포함되어 있지 않다면 '없음'으로 표시해. 
논증 요소의 정의는 다음과 같아.
주장(claim): Assertions about what exists or values that people hold. 
자료(data): Statements that are used as evidence to support the claim. 
보장(warrant): Statements that explain the relationship of the data to the claim. 
제한 조건(qualifier): Special conditions under which the claim holds true. 
뒷받침(backing): Underlying assumptions that are often not made explicit. 
반박(rebuttal): Statements that contradict either the data, warrant, backing or qualifier of an argument. 
'''

In [ ]:
def inspect_data(text):
    is_first = True
    
    try:
        text = text.strip()
        print('Original:', text)
        query_msg = query + '\nText:' + text

        # 메시지 설정하기
        messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": query_msg}
        ]

        # ChatGPT API 호출하기
        response = client.chat.completions.create(model=MODEL, messages=messages, temperature=TEMPERATURE)
        answer = response.choices[0].message.content
        answer = answer.strip()

        print(answer)
        
        return answer
        
    except openai.APIError as e:
        #Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")
        return 'Error'
        
    except openai.APIConnectionError as e:
        #Handle connection error here
        print(f"Failed to connect to OpenAI API: {e}")
        return 'Error'
        
    except openai.RateLimitError as e:
        #Handle rate limit error (we recommend using exponential backoff)
        print(f"OpenAI API request exceeded rate limit: {e}")
        return 'Error'

In [ ]:
sents = '''
I can compare the cell to a factory because cells also have their own functions and what or must to do just like in factory.
If there's a one cell or one thing that is not functioning, it will not work.
'''

In [ ]:
result = inspect_data(sents)

### Using Function Calling

In [ ]:
functions = [
    {
        "name" : "extract_element",
        "description": "Extract the elements of argumentation from the text.",
        "parameters": {
            "type": "object", 
            "properties": {
                "Claim": {
                    "type": "string",
                    "description": "Find claim from the text."
                },
                "Data": {
                    "type": "string",
                    "description": "Find data from the text."
                },
                "Warrant": {
                    "type": "string",
                    "description": "Find warrant from the text."
                },
                "Backing": {
                    "type": "string",
                    "description": "Find backing from the text."
                },
                "Qualifier": {
                    "type": "string",
                    "description": "Find qualifier from the text."
                },
                "Rebuttal": {
                    "type": "string",
                    "description": "Find rebuttal from the text."
                }
            }
        }
    }
]

In [ ]:
def inspect_data(text):
    
    start_time = time.time()
    sent = text
    sent = sent.replace('\n', ' ').strip()

    query_msg = query + '\nText:' + sent
    #query_msg = query + txt_def + '\nText:' + sent

    messages = [
            {"role": "system", "content": "너는 텍스트로부터 논증 요소를 분석하는 훌륭한 전문가야."},
            {"role": "user", "content": query_msg}
    ]
        
    response = client.chat.completions.create(model=MODEL,
                                              messages=messages,
                                              temperature=TEMPERATURE,
                                              functions=functions, 
                                              function_call = {"name": functions[0]["name"]})
    answer = response.choices[0].message.function_call.arguments
    answer = json.loads(answer)

    arguments = {}

    arguments['Answer'] = sent
    arguments['Claim'] = answer.get('Claim', '')
    arguments['Data'] = answer.get('Data', '')
    arguments['Warrant'] = answer.get('Warrant', '')
    arguments['Backing'] = answer.get('Backing', '')
    arguments['Qualifier'] = answer.get('Qualifier', '')
    arguments['Rebuttal'] = answer.get('Rebuttal', '')
    
    print('Elapsed time:', "{:.2f}".format(time.time() - start_time))
    
    return arguments

In [ ]:
result_dict = inspect_data(sents)

In [ ]:
result_dict